In [1]:
from funcoesTermosol import *
import pandas as pd
import numpy as np

In [2]:
def gauss_seidel(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u[j]
            u[i] = (f[i] - soma) / k[i, i]
        erro = ((u - u_ant) / u).max(axis=0)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

def jacobi(ite, tol, k, f):
    u = np.zeros((k.shape[0], 1))
    u_ant = np.zeros((k.shape[0], 1))
    for n in range(ite):
        for i in range(k.shape[0]): #percorre as linhas da matriz
            soma = 0
            for j in range(k.shape[1]): #percorre as colunas da matriz
                if i != j:
                    soma += k[i, j] * u_ant[j]
            u[i] = (f[i] - soma) / k[i, i]

        erro = ((u - u_ant) / u).max(axis=0)
        # print("u: ", u)
        if erro < tol:
            return u, n
        u_ant = np.copy(u)
    return u, n

In [3]:
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xls')
dados = [nn,N,nm,Inc,nc,F,nr,R] 
dados

[3,
 array([[0. , 0. , 0.3],
        [0. , 0.4, 0.4]]),
 3,
 array([[1.0e+00, 2.0e+00, 2.1e+11, 2.0e-04],
        [2.0e+00, 3.0e+00, 2.1e+11, 2.0e-04],
        [3.0e+00, 1.0e+00, 2.1e+11, 2.0e-04]]),
 2,
 array([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [ 150.],
        [-100.]]),
 3,
 array([[0.],
        [2.],
        [3.]])]

In [4]:
# Criação da matriz de rigidez global

Kg = np.zeros((2*nn,2*nn))

for i in range(nm):
    n1 = int(Inc[i][0])
    n2 = int(Inc[i][1])
    x_n1 = N[0][n1-1]
    y_n1 = N[1][n1-1]

    x_n2 = N[0][n2-1]
    y_n2 = N[1][n2-1]

    L = np.sqrt((x_n2-x_n1)**2 + (y_n2-y_n1)**2)
    c = (x_n2-x_n1)/L
    s = (y_n2-y_n1)/L

    E = Inc[i][2]
    A = Inc[i][3]

    k = (E*A)/L * np.array([[c**2, c*s, -c**2, -c*s],  # matriz de rigidez local
                            [c*s, s**2, -c*s, -s**2],
                            [-c**2, -c*s, c**2, c*s],
                            [-c*s, -s**2, c*s, s**2]])

    k = np.array(k)

    indices = sorted([2*n1-2, 2*n1-1, 2*n2-2, 2*n2-1])

    for i in range(4):
        for j in range(4):
            Kg[indices[i], indices[j]] += k[i, j]
    
Kg
     


array([[ 3.0240e+07,  4.0320e+07,  0.0000e+00,  0.0000e+00, -3.0240e+07,
        -4.0320e+07],
       [ 4.0320e+07,  1.5876e+08,  0.0000e+00, -1.0500e+08, -4.0320e+07,
        -5.3760e+07],
       [ 0.0000e+00,  0.0000e+00,  1.4000e+08,  0.0000e+00, -1.4000e+08,
         0.0000e+00],
       [ 0.0000e+00, -1.0500e+08,  0.0000e+00,  1.0500e+08,  0.0000e+00,
         0.0000e+00],
       [-3.0240e+07, -4.0320e+07, -1.4000e+08,  0.0000e+00,  1.7024e+08,
         4.0320e+07],
       [-4.0320e+07, -5.3760e+07,  0.0000e+00,  0.0000e+00,  4.0320e+07,
         5.3760e+07]])

In [5]:
# Remover linhas dos graus de liberdade restritos
R = [int(i[0]) for i in R.tolist()]
Kg = np.delete(Kg, R, axis=0)
F = np.delete(F, R, axis=0)

# Remover colunas dos graus de liberdade restritos
Kg = np.delete(Kg, R, axis=1)

Kg, F

(array([[ 1.5876e+08, -4.0320e+07, -5.3760e+07],
        [-4.0320e+07,  1.7024e+08,  4.0320e+07],
        [-5.3760e+07,  4.0320e+07,  5.3760e+07]]),
 array([[   0.],
        [ 150.],
        [-100.]]))

In [6]:
# Resolver o sistema de equações para obter os deslocamentos nodais
ite = 100
tol = 1e-10
u_j, n_j = jacobi(ite, tol, Kg, F)
u_gs, n_gs = gauss_seidel(ite, tol, Kg, F)

print("Método de Jacobi")
print(u_j, n_j)
print("-----------------------------")
print("Método de Gauss-Seidel")
print(u_gs, n_gs)

Método de Jacobi
[[-9.52380958e-07]
 [ 1.60714286e-06]
 [-4.01785713e-06]] 99
-----------------------------
Método de Gauss-Seidel
[[-9.52380952e-07]
 [ 1.60714286e-06]
 [-4.01785714e-06]] 22


C:\Users\arthu\AppData\Local\Temp\ipykernel_26824\726755875.py:28: RuntimeWarning: invalid value encountered in divide
  erro = ((u - u_ant) / u).max(axis=0)
C:\Users\arthu\AppData\Local\Temp\ipykernel_26824\726755875.py:11: RuntimeWarning: invalid value encountered in divide
  erro = ((u - u_ant) / u).max(axis=0)


In [7]:
# Calcular reações de apoio	


In [8]:
# Deformações

In [9]:
# Forças internas

In [10]:
# Tensões internas